# Basic usage of `bs.py`

In [1]:
%matplotlib notebook
%run bs.py
plt.rcParams['animation.html'] = 'jshtml' # 'html5' requires FFmpeg
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2})
sns.set_palette(sns.color_palette("Set2", 4))

## Class `Population`

A instance of the class `Population` contains a frequency distribution of organisms over a discrete set of Malthusian growth rates, and provides a method for annual update of the distribution. The instance is defined by an initial frequency distribution over growth rates, along with a probability distribution over the effects of mutation on the growth rates of offspring. 

### Instantiation

Blah-blah

In [ ]:
birth_redistribution = GaussianEffects()
initial_distribution = GaussianRates(birth_redistribution.n_rates)
population = Population(initial_distribution, birth_redistribution, n_updates_per_year=1)

See the section below on class ``Distribution`` for more information on the distributions over growth rates and mutation effects.

**Need explanation of updates per year**.

The initial frequency distribution over growth rates and the probability distribution *changes* in growth rate (mutation effects) must agree in the range of discrete growth rates. It is easier to maintain consistency with the BS experiments by associating default values with the latter than with the former.

### Subclass `BS_Population`

The class `BS_Population` replaces the annual update method of its superclass `Population` with one that agrees with the BS numerical experiments. The annual update method of a `Population` instance treats the birth rates and the death rate as logarithmic. It also lumps the out-of-range births (mutants) with the endpoints of the population distribution. The annual update method of a `BS_Population` instance treats the rates as linear, and discards the out-of-range births, precisely as Basener's script does.

By default, a `BS_Population` is what BS call &ldquo;infinite,&rdquo; meaning that relatively small frequencies are retained, just as with a `Population`.

## Class `Evolution`

An instance of the class `Evolution` records the `trajectory` of a `Population` in the space of frequency distributions. Each step in the `trajectory` corresponds to one annual update of the the `Population` object. This makes it easy to address the question of how the number of updates per year affects the trajectory. The `Evolution` object provides a method for extending the `trajectory` by a given number of years, along with methods for access and analysis of the `trajectory`.

### Instantiation

Blah-blah

In [ ]:
evolution = Evolution(population)

The length of a Population instance is the number of discrete growth rates (fitnesses) over which the population is distributed.

In [ ]:
bs_population = BS_Population(initial_population, birth_redistribution)
bs_evolution = Evolution(bs_population)

In [ ]:
len(population), len(bs_population)

The length of an `Evolution` instance is the number of years in the evolutionary process. The length of a fresh instance is 1 because year 0 is included.

In [ ]:
len(evolution), len(bs_evolution)

To extend the evolutionary process by $n$ years, call the `Evolution` instance with parameter $n.$

In [ ]:
evolution(100), bs_evolution(100)
len(evolution), len(bs_evolution)

In [ ]:
evolution(200), bs_evolution(200)
len(evolution), len(bs_evolution)

Now we animate the two evolutionary processes. The `stride` parameter determines how many years the animation advances in each frame. The time required to generate an HTML5 video is inversely proportional to the stride.

(I haven't yet provided the means to determine which of the differences in `Population` and `BS_Population` is making a difference, but I'm pretty sure that dicarding out-of-range mutants has little effect in this particular experiment. The further the birth rates from 0, the worse the linear approximation.)

In [ ]:
animate([evolution, bs_evolution], ['TE', 'BS'], stride=2, subtitle='\nGaussian Mutation Effects')

Now we switch to the (reflected, rescaled) Gamma distribution over mutation effects&nbsp;&mdash; but without Basener's gimmick of setting the probability that mutation has no effect on fitness to the probability that mutation has a minimally deleterious effect on fitness. We want to see the change in the evolutionary process when mutations at $L=2^n$ loci, $n = 0, 1, 2, 14,$ have iid effects on fitness, with the number of mutations per offspring held at 1.

In [ ]:
# Pixiedust Node fails if the objects are stored in lists
exponents = np.array([0, 1, 2, 14])
iid_mutations = np.empty(len(exponents), dtype=object)
processes = np.empty(len(exponents), dtype=object)

for i in range(len(exponents)):
    iid_mutations[i] = GammaEffects(number_of_mutations=1, log_number_of_loci=exponents[i])
    initial_population = GaussianRates(iid_mutations[i].n_rates)
    processes[i] = Evolution(Population(initial_population, iid_mutations[i]))

We run each of the evolutionary processes for 2500 years.

In [ ]:
for p in processes:
    p(2500)

Now we animate the processes.

In [ ]:
animate(processes, exponents, stride=10, subtitle='\nOne Mutation, Log-Number of Loci Varying')

To get a clearer picture, we omit $n=2$ from the animation.

In [ ]:
p = processes[[0, 1, 3]]
e = exponents[[0, 1, 3]]
animate(p, e, stride=10, subtitle='\nOne Mutation, Log-Number of Loci Varying')

Let's plot the distributions of mutation effects for $\log_2 L = 0, 1, 14.$

In [ ]:
fig = plt.figure()
axes = fig.gca()
line_width = 1
offset = 0.00010
line = iid_mutations[0].vlines(axes, x_offset=-offset, label=exponents[0])
line.set(color='r', lw=line_width)
line = iid_mutations[1].vlines(axes, x_offset= 0.0000, label=exponents[1])
line.set(color='b', lw=line_width)
line = iid_mutations[3].vlines(axes, x_offset=+offset, label=exponents[3])
line.set(color='g', lw=line_width)
plt.grid()
plt.xlim(-0.005, +0.005)
plt.legend(loc='best')
plt.title('Gamma Mutation Effects\nOne Mutation, Log-Number of Loci Varying');

Now we take a look at the upper tail.

In [ ]:
axes.set_ylim(0, 0.0005)
fig

In [ ]:
%%node

var PctBeneficial = 0.001;
var AnimateId;


function Gamma(Z) {
    with(Math) {
        var S = 1 + 76.18009173 / Z - 86.50532033 / (Z + 1) + 24.01409822 / (Z + 2) - 1.231739516 / (Z + 3) + .00120858003 / (Z + 4) - .00000536382 / (Z + 5);
        var G = exp((Z - .5) * log(Z + 4.5) - (Z + 4.5) + log(S * 2.50662827465));
    }
    return G
}


function mutationProb(mDiff, mDelta, mt) {
    if (mt == "Gaussian") {
        var stdevMutation = /* 0.0005 */ 0.002;  //# 0.002 in BS article
        GaussianMultiplicativeTerm = 1 / (stdevMutation * Math.sqrt(2 * Math.PI));
        f = GaussianMultiplicativeTerm * Math.exp(-0.5 * Math.pow((mDiff) / stdevMutation, 2));
        f = f * mDelta;
    }
    if (mt == "Asymmetrical Gaussian") {
        var stdevMutation = 0.001;
        var stdevMutationBeneficial = 0.001;
        if (mDiff > 0) {
            GaussianMultiplicativeTerm = 1 / (stdevMutationBeneficial * Math.sqrt(2 * Math.PI));
            f = ((PctBeneficial) / 0.5) * GaussianMultiplicativeTerm * Math.exp(-0.5 * Math.pow((mDiff) / stdevMutationBeneficial, 2));
        }
        else {
            GaussianMultiplicativeTerm = 1 / (stdevMutation * Math.sqrt(2 * Math.PI));
            f = ((1 - PctBeneficial) / 0.5) * GaussianMultiplicativeTerm * Math.exp(-0.5 * Math.pow((mDiff) / stdevMutation, 2));
        }
        f = f * mDelta;
    }
    if (mt == "Gamma") {
        if (mDiff == 0) mDiff = -mDelta;
        var sBarBeneficial = 0.001;
        var sBarDeleterious = 0.001;
        var aBeneficial = 0.5;
        var aDeleterious = 0.5;
        var bBeneficial = aBeneficial / sBarBeneficial;
        var bDeleterious = aDeleterious / sBarDeleterious;
        if (mDiff > 0) f = (PctBeneficial) * Math.pow(bBeneficial, aBeneficial) * Math.pow(mDiff, aBeneficial - 1) * Math.exp(-bBeneficial * mDiff) / Gamma(aBeneficial);
        if (mDiff < 0) f = (1 - PctBeneficial) * Math.pow(bDeleterious, aDeleterious) * Math.pow(Math.abs(mDiff), aDeleterious - 1) * Math.exp(-bDeleterious * Math.abs(mDiff)) / Gamma(aDeleterious);
        f = f * mDelta;
    }
    if (mt == "None" || mt == "NoneExact") {
        f = 0;
        if (mDiff == 0) f = 1;
    }
    return f;
}


function runSimulation(pctBeneficial = 0.001,  //# "Percentage of mutations that are beneficial"
                       mt = "Gamma",           //# "Mutation Distribution Type": Gamma, Gaussian, None, or NoneExact
                       PopSize = "Finite",     //# "Population Size" is Finite or Infinite
                       numYears = 3500,        //# "Number of years" in the evolutionary process
                       numIncrements = 500)    //# "Number of Discrete Population Fitness Values"
{
    //# Moved variables set by form inputs into parameter list
    console.log('initializing');
    PctBeneficial = pctBeneficial; //# set global variable
    var maxPopulationSize = 10 ^ 9;
    var mean = 0.044;
    var stdev = 0.005;
    var fitnessRange = [-0.1, 0.15];
    var numStDev = 11.2;
    var deathRate = 0.1;
    var meanFitness = new Array(numYears);
    var varianceFitness = new Array(numYears);
    var P = new Array(numIncrements);
    var MeanROC = new Array(numYears);
    var MVRatio = new Array(numYears);
    var MVDiff = new Array(numYears);
    var yearVariable = new Array(numYears);
    var minFitness = mean - numStDev * stdev;
    var maxFitness = mean + numStDev * stdev;
    var m = new Array(numIncrements);
    mDelta = (fitnessRange[1] - fitnessRange[0]) / (numIncrements);
    m[0] = fitnessRange[0];
    for (i = 1; i < numIncrements; i++) {
        m[i] = fitnessRange[0] + i * mDelta;
    }
    var b = new Array(numIncrements);
    for (i = 0; i < numIncrements; i++) {
        b[i] = m[i] + deathRate;
    }
    GaussianMultiplicativeTerm = 1 / (stdev * Math.sqrt(2 * Math.PI));
    //# Correct error: last iteration assigns to memory out of bounds
    for (i = 0; i < numIncrements /* + 1 */; i++) { 
        P[i] = GaussianMultiplicativeTerm * Math.exp(-0.5 * Math.pow((m[i] - mean) / stdev, 2));
        if (m[i] < minFitness) {
            P[i] = 0
        };
        if (m[i] > maxFitness) {
            P[i] = 0
        };
    }
    var Psolution = new Array(numYears);
    var PsolutionForPlot = new Array(numYears);
    for (var t = 0; t < numYears; t++) {
        Psolution[t] = new Array(numIncrements);
        PsolutionForPlot[t] = new Array(numIncrements);
    }
    s = 0;
    for (i = 0; i < numIncrements; i++) {
        s = s + P[i]
    }
    var maxPinitial = 0;
    for (var i = 0; i < numIncrements; i++) {
        Psolution[0][i] = P[i] / s;
        PsolutionForPlot[0][i] = P[i] / s;
        maxPinitial = Math.max(maxPinitial, PsolutionForPlot[0][i]);
    }
    var MP = new Array(numIncrements);
    for (i = 0; i < numIncrements; i++) {
        MP[i] = new Array(numIncrements);
        for (j = 0; j < numIncrements; j++) {
            MP[i][j] = b[j] * mutationProb(m[i] - m[j], mDelta, mt);
        }
    }
    meanFitness[0] = mean;
    varianceFitness[0] = 0;
    for (var i = 0; i < numIncrements; i++) {
        varianceFitness[0] = varianceFitness[0] + (m[i] - mean) * (m[i] - mean) * Psolution[0][i];
    }
    console.log('iterating');
    for (t = 1; t < numYears; t++) {
        s = 0;
        meanFitness[t] = 0;
        varianceFitness[t] = 0;
        for (i = 0; i < numIncrements; i++) {
            Psolution[t][i] = Psolution[t - 1][i];
            for (j = 0; j < numIncrements; j++) {
                Psolution[t][i] = Psolution[t][i] + Psolution[t - 1][j] * MP[i][j];
            }
            Psolution[t][i] = Psolution[t][i] - deathRate * Psolution[t - 1][i];
            if (mt == "NoneExact") Psolution[t][i] = Psolution[0][i] * Math.exp(t * m[i]);
            s = s + Psolution[t][i];
        }
        if (PopSize == "Finite") {
            maximumP = Math.max.apply(Math, Psolution[t]);
            for (i = 0; i < numIncrements; i++) {
                Psolution[t][i] = Psolution[t][i] * (Psolution[t][i] > maximumP * 0.000000001);
            }
        }
        for (var i = 0; i < numIncrements; i++) {
            PsolutionForPlot[t][i] = Psolution[t][i] / s;
            meanFitness[t] = meanFitness[t] + m[i] * PsolutionForPlot[t][i];
        }
        for (var i = 0; i < numIncrements; i++) {
            mMinusMean = (m[i] - meanFitness[t]);
            varianceFitness[t] = varianceFitness[t] + mMinusMean * mMinusMean * PsolutionForPlot[t][i];
        }
    }
    console.log('done');
    return {'trajectory' : Psolution, 'growth_rates' : m}
}

In [ ]:
%%node
bs_result = runSimulation(pctBeneficial = 0.001,  //# "Percentage of mutations that are beneficial"
                          mt = "Gaussian",        //# "Mutation Distribution Type": Gamma, Gaussian, None, or NoneExact
                          PopSize = "Finite",     //# "Population Size" is Finite or Infinite
                          numYears = 300,         //# "Number of years" in the evolutionary process
                          numIncrements = 250);   //# "Number of Discrete Population Fitness Values"

In [ ]:
bs = WrappedTrajectory(bs_result['trajectory'], bs_result['growth_rates'])
bs_alt = Evolution(BS_Population(GaussianRates(251), GaussianEffects(251), threshold_norm=np.max))
bs_alt(len(bs) - 1)
len(bs), len(bs_alt)

In [ ]:
mean_variance_plots([bs, bs_alt], ['BS', 'Alt'], subtitle='Do we match BS?')

In [ ]:
animate([bs, bs_alt], ['BS', 'Alt'], stride=3, subtitle=': Do we match BS?')

In [ ]:
(bs.annual_growth_rates[:] == bs_alt.annual_growth_rates[:-1]).all()

In [ ]:
np.sum(np.abs(bs[0] - bs_alt[0][:-1]))